In [ ]:
import pickle

with open('data/labeled_df.pkl', 'rb') as f:
    labeled_df = pickle.load(f)

with open('data/pseudo_labeled_df.pkl', 'rb') as f:
    unlabeled_df = pickle.load(f)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Separate features and labels
X_labeled = labeled_df.drop(columns=["label"])
y_labeled = labeled_df["label"]

X_unlabeled = unlabeled_df.drop(columns=["label"])
y_pseudo = unlabeled_df["label"]  # pseudo-labels

# Train model on true labeled data
model1 = RandomForestClassifier(random_state=42)
model1.fit(X_labeled, y_labeled)

# Predict on unlabeled (pseudo-labeled) data
y_pred_unlabeled = model1.predict(X_unlabeled)

# Compare to pseudo-labels
print("🔍 Step 1: True-label model vs Pseudo-labels")
print(classification_report(y_pseudo, y_pred_unlabeled))


In [ ]:
# Train on pseudo-labeled data
model2 = RandomForestClassifier(random_state=42)
model2.fit(X_unlabeled, y_pseudo)

# Predict on labeled data
y_pred_labeled = model2.predict(X_labeled)

# Compare to true labels
print("🔁 Step 2: Pseudo-label model vs True labels")
print(classification_report(y_labeled, y_pred_labeled))
